In [1]:
import matplotlib.pyplot as plt
%matplotlib notebook
from ipywidgets import widgets
import IPython.display as display

In [19]:
#efficiency = 0.9
#vin = 5.0
#vout = 3.3
#iout = 1.4
#freq = 1e6
#vovershoot = vout * 0.05
#vripple = vout * 0.01

vin = widgets.FloatText(value=5.0, description='Vin (V)')
vout = widgets.FloatText(value=3.3, description='Vout (V)')
iout = widgets.FloatText(value=1.4, description='Iout (A)')
freq = widgets.FloatText(value=1e6, description='Freq (Hz)')
efficiency = widgets.FloatText(value=0.9, description='Efficiency')
istep = widgets.FloatText(value=1.4, description='Load step (A)')
vin_overshoot = widgets.FloatText(value=100e-3, description='Vin Overshoot (V)')
vin_ripple = widgets.FloatText(value=30e-3, description='Vin Ripple (V)')
vout_overshoot = widgets.FloatText(value=100e-3, description='Vout Overshoot (V)')
vout_ripple = widgets.FloatText(value=30e-3, description='Vout Ripple (V)')
out1 = widgets.Output()
L = widgets.FloatText(value=2.2e-6, description='L (H)')
out2 = widgets.Output()
R_L = widgets.FloatText(value=102e-3, description='L DCR (Ohm)')
out3 = widgets.Output()

def update(sender):
    out1.clear_output()
    with out1:
        duty = vout.value/vin.value/efficiency.value
        ton = duty/freq.value
        print('duty cycle: %.2f' % duty)
    
        ripple_fact = 0.4
        L_min = vin.value / (4 * freq.value * ripple_fact * vout.value)
        print('suggested L_min: %.2fuH' % (L_min * 1e6))

    out2.clear_output()
    with out2:
        i_L = (vin.value-vout.value)/L.value*ton
        print('ripple current: %.2gA' % i_L)
        
        ipeak = iout.value + i_L/2
        print('peak current: %.2fA' % ipeak)
        iout_rms = (iout.value**2 + i_L**2/12)**0.5
        print('rms output current: %.2fA' % iout_rms)
        
        print('rated inductor current: %.1fA' % (ipeak * 1.2))
        
    out3.clear_output()
    with out3:
        P_L = iout_rms**2 * R_L.value
        print('copper loss: %.2fW, %.0f%%' % (P_L, P_L/(iout_rms*vout.value)*100))
        print('')
        
        # from http://www.ti.com/lit/an/slyt670/slyt670.pdf
        Cin_min_ripple = duty*(1-duty)*iout.value/(vin_ripple.value*freq.value)
        print('min. input capacitance for %.2gV ripple voltage: %.1fuF' % (vin_ripple.value, Cin_min_ripple * 1e6))
        vin_ripple_real = duty*(1-duty)*iout.value/(Cin_min_ripple*freq.value)
        print('input ripple voltage at min. capacitance: %.2fV' % vin_ripple_real)
    
        control_bw = 20e3 #20kHz control BW - XXX depends on previous stage
        trise = 1/(control_bw*4)
        Cin_min_bulk = istep.value/2*duty*trise/vin_overshoot.value - Cin_min_ripple
        print('min. bulk input capacitance for %.2fV transient response to %.1fA load step: %.1fuF' % (vin_overshoot.value, istep.value, Cin_min_bulk * 1e6))
        Cin_ESR = vin_overshoot.value/(iout.value * duty)
        print('max. bulk ESR: %.2fOhm' % Cin_ESR)
        ibulk_rms = 1/(2*3**0.5)*vin_ripple_real/Cin_ESR
        print('min. bulk ripple current rating: %.2fA' % ibulk_rms)
        #Cout_min = max(Cout_min_ripple, Cout_min_transient)
        #print('min input capacitance: %.1fuF' % (Cin_min * 1e6))
        print('')

        # from http://www.onsemi.com/pub/Collateral/AND9135-D.PDF
        Cout_min_transient = L.value * istep.value**2/((vout.value + vout_overshoot.value)**2 - vout.value**2)
        print('min. output capacitance for %.2fV transient response to %.1fA load step: %.1fuF' % (vout_overshoot.value, istep.value, Cout_min_transient * 1e6))
        Cout_min_ripple = i_L / (8 * freq.value * vout_ripple.value)
        print('min. output capacitance for %.2gV ripple voltage: %.1fuF' % (vout_ripple.value, Cout_min_ripple * 1e6))
        Cout_min = max(Cout_min_ripple, Cout_min_transient)
        print('output transient overshoot at min. capacitance: %.2fV'% ((L.value*istep.value**2/Cout_min + vout.value**2)**0.5-vout.value))
        print('output ripple voltage at min. capacitance: %.2fV' % (i_L/(8*freq.value*Cout_min)))
        print('min output capacitance: %.1fuF' % (Cout_min * 1e6))

for w in (vin, vout, iout, freq, efficiency, istep, vin_overshoot, vin_ripple, vout_overshoot, vout_ripple, L, R_L):
    w.observe(update)
   
display.display(vin, vout, iout, freq, efficiency, istep, vin_overshoot, vin_ripple, vout_overshoot, vout_ripple, out1, L, out2, R_L, out3)
update(None)